In [1]:
#  — setup
import socketserver, threading, csv, datetime
from pathlib import Path

LOG_FILE = Path('honeypot_log.csv')

# create header if not exists
if not LOG_FILE.exists():
    with LOG_FILE.open('w', newline='') as f:
        csv.writer(f).writerow(['timestamp','src_ip','src_port','recv_text','recv_hex','banner_sent'])
    print("Created honeypot_log.csv")
else:
    print("honeypot_log.csv already exists")


honeypot_log.csv already exists


In [2]:
import socketserver
import csv
from datetime import datetime, timezone

# --- handler class ---
class HoneyPotHandler(socketserver.BaseRequestHandler):
    def handle(self):
        peer_ip, peer_port = self.client_address
        try:
            self.request.settimeout(2.0)
            data = self.request.recv(4096)
        except Exception:
            data = b''

        recv_hex = data.hex()
        recv_text = data.decode('utf-8', errors='replace').strip()

        # fake banner (mimics SSH to attract scanners)
        banner = b"SSH-2.0-OpenSSH_7.6p1 Ubuntu-4ubuntu0.3\r\n"
        try:
            self.request.sendall(banner)
        except Exception:
            pass
        timestamp = datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")

        # append to csv
        with LOG_FILE.open('a', newline='') as f:
            csv.writer(f).writerow([
                timestamp,
                peer_ip,
                peer_port,
                recv_text,
                recv_hex,
                banner.decode('ascii', 'ignore').strip()
            ])

        # quick console feedback
        print(f"[{timestamp}] {peer_ip}:{peer_port} -> {recv_text or '[no data]'}")


class ThreadedTCPServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    allow_reuse_address = True


In [4]:
#  — start server (non-blocking)
HOST = '127.0.0.1'   
PORT = 2222          

server = ThreadedTCPServer((HOST, PORT), HoneyPotHandler)
t = threading.Thread(target=server.serve_forever, daemon=True)
t.start()
print(f"Honeypot listening on {HOST}:{PORT} — server thread started.")
print("To stop: run server.shutdown() in a later cell.")


Honeypot listening on 127.0.0.1:2222 — server thread started.
To stop: run server.shutdown() in a later cell.


In [5]:
import socket, time
s = socket.socket()
s.settimeout(3)
s.connect((HOST, PORT))
s.send(b'HELLO_FROM_TEST_CLIENT\r\n')
try:
    data = s.recv(1024)
    print("Banner received:", data)
except Exception as e:
    print("No banner or timeout:", e)
s.close()
time.sleep(0.2)  # let handler log


Banner received: b'SSH-2.0-OpenSSH_7.6p1 Ubuntu-4ubuntu0.3\r\n'
[2025-09-23T13:29:17.860898Z] 127.0.0.1:62891 -> HELLO_FROM_TEST_CLIENT


In [5]:

import pandas as pd
df = pd.read_csv('honeypot_log.csv')
df.tail(20)


,timestamp,src_ip,src_port,recv_text,recv_hex,banner_sent
0,2025-09-18T08:16:30.265316Z,127.0.0.1,64276,HELLO_FROM_TEST_CLIENT,48454c4c4f5f46524f4d5f544553545f434c49454e540d0a,SSH-2.0-OpenSSH_7.6p1 Ubuntu-4ubuntu0.3


In [6]:
# — stop server cleanly
server.shutdown()
server.server_close()
print("Honeypot stopped.")


Honeypot stopped.
